# Lesson 4: 


### Import the required libraries

In [ ]:
import face_recognition as fr
from matplotlib import pyplot as plt
plt.style.use('default')
import os
import cv2
import numpy as np
from datetime import datetime

### Read and plot the image

In [ ]:
path = r'..\.sample_photos' #path to the sample_photos folder
image = plt.imread(os.path.join(path, 'samplephoto1.jpg')) #read the image

plt.xticks([], [])
plt.yticks([], [])
plt.imshow(image)
plt.show()

### Face Detection

In [ ]:
from datetime import datetime
start = datetime.now()
face_locations =  fr.api.face_locations(image,  number_of_times_to_upsample=1, model='cnn') #detect face locations, try different models and no. of times to upsample
print('Time passed:', datetime.now() - start)

In [ ]:
print('No. of faces found: %i' %len(face_locations))
print('Face locations:', face_locations)

### Drawing bounding boxes and plot the image

In [ ]:
new_image = np.array(image) #declare a new array to draw on

#draw bounding boxes around the faces in the new_image array
for top, right, bottom, left in face_locations: 
        #Draw bounding boxes on new_image for each locations
    
#Plotting the image    
plt.xticks([], [])
plt.yticks([], [])
plt.imshow(new_image)
plt.show()

# Questions
    

1. Try to detect all the faces out of the given photos
2. How many faces are there in each photo?
3. Have all the faces been detected?
4. How long does it take for the detection?
5. Try different models ('hog' or 'cnn') and differnt number_of_times_to_upsample, compare the performance (percentage of faces it is able to detect and the time taken)
Hints:
- Keep number_of_times_to_upsample < 4 when using 'hog' model or < 3 when using 'cnn' model, you may get memory error or cause your computer to hang if this number is too large.
- You may count the time with the datetime.now() function

## Exercise to understand the Histogram of Oriented Gradients (HOG) feature descriptor

In [ ]:
from matplotlib import pyplot as plt
plt.style.use('default')
import numpy as np
import os
import cv2

## Loading and cropping the image

In [ ]:
path = r'path_to_the_folder_containing_fence.jpg' #path to the folder containing the fence.jpg file
file = 'fence.jpg'

img = plt.imread(os.path.join(path, file)) #read in the image
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert it to black and white
small_img = img_gray[100:132,100:164] #crop the image
small_img = small_img.astype('float')

You may rotate the image with the np.rot90() function

In [ ]:
#small_img = np.rot90(small_img) #rotate the image by 90 degrees
plt.imshow(small_img, cmap='gray')
plt.title('Input image')
plt.savefig('small_img.jpg',dpi=1000)
plt.show()

## Background math

### Vectors

In physics, vectors are quantities with both the magnitude and the direction, examples are displacement, velocity, acceleration, force etc.

Algebraically, a vector can be represented by a tuple of its components, i.e. for a 2-dimensionally vector $\vec{r}$:

$$
\vec{r} = (r_x, r_y)
$$

where $r_x$ and $r_y$ are the components of $\vec{r}$ along the $x$ and $y$ axis respectively.

The Euclidean norm (or the magnitude) of a vector $|\vec{r}|$ can be found by the Pythagoras Theorem:

$$
|\vec{r}| = \sqrt{r_x^2+r_y^2}
$$

and the direction $\theta$ can be found by:

$$
\begin{align}
\tan \theta &= \frac{r_y}{r_x} \\
\theta &= \arctan (\frac{r_y}{r_x})
\end{align}
$$

A unit vector $\hat r$ is a vector with a magnitude of 1, it is obtained by dividing each components of a vector $\vec{r}$ by its norm, i.e.:

$$
\hat r = ( \frac{r_x}{|\vec{r}|}, \frac{r_y}{|\vec{r}|} ) = \frac{1}{|\vec{r}|} (r_x, r_y)
$$

In general, the concept of vectors can be generalized to an n-dimensional space, i.e. a vector with $n$ components. In face recognition, a face identity is represented an n-dimensional vector called the face embedding, we will discuss it later.

### Example:

If
$$\vec{r} = (2, 3)$$

then:
1. 
$$
|\vec{r}| = \sqrt{2^2+3^2} = \sqrt{13},
$$
<br>

2. 
$$
\begin{align}
\tan \theta &= \frac{3}{2} \\
\theta &= \arctan (\frac{3}{2}) \\
& \approx 56.3^\circ,
\end{align}
$$
<br>

3. 
$$
\hat r = \frac{1}{\sqrt{13}} (2, 3)
$$

In [ ]:
from matplotlib.patches import Arc

angle_plot = Arc(xy=(0,0), width=1.2, height=1.5, theta1=0, theta2=56.3, color='black')
ax = plt.gca()
ax.add_patch(angle_plot)
plt.xlim(0,4)
plt.ylim(0,4)
plt.grid()
plt.title('Example of a vector', fontsize=18,  y=1.03)
plt.xlabel('x', fontsize=15,labelpad=10)
plt.ylabel('y', fontsize=15, rotation=0,labelpad=20)
plt.annotate(r'$\vec{r}$', xy=(1.1,2.1), fontsize=20)
plt.annotate('(2,3)', xy=(2,3), fontsize=15)
plt.annotate(r'$\theta = arctan( \frac{3}{2} ) \approx 56.3^\circ$', xy=(0.6,0.3), fontsize=15)
plt.quiver(*(0,0), 2, 3,  angles='xy', scale_units='xy', scale=1, color='blue')
plt.show()

### Derivatives

The derivative of a function $f(x)$ is defined by the following limit:


$$
    \frac{d f(x)}{dx} = \lim_{\Delta x\to 0} \frac{f(x+\Delta x) - f(x)}{\Delta x}
$$



It can be shown that if $f(x)=x^n$ for any constant $n$, then $\frac{df(x)}{dx}=n x^{n-1}$.



<br>
    <center>
        <img src="slope.jpg" width="220" height="20" />   
        <figcaption >Fig.1 - Slope of a linear function.</figcaption>
    </center>
</br> 
<br>    
    <center>  
        <img src="derivative.jpg" width="320" height="320" />   
        <figcaption >Fig.2 - Derivative of a function.</figcaption>    
    </center>
</br>

### Partial derivatives and gradient

The partial derivatives of a multivariate function $f(x_1,x_2,\ldots,x_k)$ are defined by the limit:

$$
    \frac{\partial f(x_1,x_2,\ldots,x_k)}{\partial{x_i}} = \lim_{\Delta x_i\to 0} \frac{f(x_1,\ldots, x_i+\Delta x_i,\ldots, x_k) - f(x_1,\ldots,x_i,\ldots,x_k)}{\Delta x_i}
$$

and the gradient of $f(x_1,x_2,\ldots,x_k)$ and its norm are given by:

$$
\vec{\nabla} f(x_1,x_2,\ldots,x_k) = ( \frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2}, \ldots,\frac{\partial f}{\partial x_k} )
$$ 
, and 
$$
|\vec{\nabla} f(x_1,x_2,\ldots,x_k)| = \sqrt{ (\frac{\partial f}{\partial x_1})^2 + (\frac{\partial f}{\partial x_2})^2, \ldots, (\frac{\partial f}{\partial x_k})^2}
$$ respectively.

For example, if $f(x,y) = x^3 + y^4$, the partial derivatives are:

$$
\frac{\partial f(x,y)}{\partial{x}} = 3x^{2}
$$

$$
\frac{\partial f(x,y)}{\partial{y}} = 4y^{3}
$$

The gradient and its norm are:

$$
\vec{\nabla} f(x,y) = (3x^{2}, 4y^{3})
$$
, and 
$$
|\vec{\nabla} f(x,y)| = \sqrt{(3x^{2})^2+(4y^{3})^2} = \sqrt{ 9x^4 + 16y^6 }
$$
respectively.


## Convolution

The convolution operation is important in computer vision for edge detection and a type of neural network called convolutional neural network. The convolution between the matrices $f$ and $g$ is denoted by $f \ast g$, and it is defined by the following formula:

$$
(f \ast g)(x,y) = \sum_{u=-\infty}^{+\infty} \sum_{v=-\infty}^{+\infty} f(u, v) g(x-u,y-v)
$$



For example:
    
<br>
<center>
<img src="convolution.jpg" width="420" height="420" />    
<\center>
<br>
<br>   
    
The matrix f can be an image and the matrix g is called the filter or the kernel. Sometimes, 0s are padded around the border of the matrix f to make the size of the output equals to the size of the matrix f, it is called the 'same padding'.

## Calculating image gradients using convolution

For dicretized data (i.e. an image), we can approximate $\frac{\partial f(x,y)}{\partial x}$ by taking $\Delta x = 1$ in the limit, where $f(x,y)$ represents the pixel value at the position $(x,y)$. It corresponds to **the change of pixel value when moving by 1 pixel along the x-axis**. It can be showned that it is equivalent to the following convolution operation:

$$
\begin{equation}
\frac{\partial f(x,y)}{\partial x}  \approx \frac{f(x+1,y) - f(x,y)}{1} \\
                                     = f \ast \begin{pmatrix} -1 & 1 \end{pmatrix}
\end{equation}
$$

Similarly, $\frac{\partial f(x,y)}{\partial y}$ can be approximated by taking $\Delta y = 1$ in the limit, it corresponds to **the change of pixel value when moving by 1 pixel along the y-axis**, and it is equivalent to the following convolution operation:

$$
\begin{equation}
\frac{\partial f(x,y)}{\partial y} \approx \frac{f(x,y+1) - f(x,y)}{1} \\
                                    = f \ast \begin{pmatrix} -1 \\ 1 \end{pmatrix}
\end{equation}
$$

The matrix $\begin{pmatrix} -1 & 1 \end{pmatrix}$ is a vertical line detector and the matrix $\begin{pmatrix} -1 \\ 1 \end{pmatrix}$ is a horizontal line detector.

## Code

import the convolve2d() function from scipy.signal module, rename it as conv2d()

In [ ]:
from scipy.signal import convolve2d as conv2d

Define the vertical line and horizontal line detectors as numpy arrays with the variable names vline and hline respectively

In [ ]:
vline =  #vertical line detectors
hline =  #horizontal line detectors

vline = vline.reshape(1,-1)
hline = hline.reshape(-1,1)

print('vline:\n',vline)
print('hline:\n',hline)

Compute the convolution of the input image with the vertical line detector and plot out the resulting image, how does it look like?

Hint: Use the conv2d() function and specify mode='same', documentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html

In [ ]:
dx =  #convolution with vline filter, use 'same' padding (partial derivative along x axis)

plt.title('Convolution with vline detector')
plt.imshow(dx, cmap='gray')
plt.show()

Compute the convolution of the input image with the horizontal line detector and plot out the resulting image, how does it look like?

In [ ]:
dy =  ##convolution with hline filter, use 'same' padding (partial derivative along y axis)

plt.title('Convolution with hline detector')
plt.imshow(dy, cmap='gray')
plt.show()

# Question
Can you explain what you have seen in the resulting images?

## Calculating gradient magnitude (GM) and gradient direction (GD) 

The gradient magnitude (GM) is the norm of the gradient,and the gradient direction (GD) is the angle between the gradient vector and the x-axis.

Calculate the gradient magnitude by:
$$
GM = |\vec{\nabla} f(x,y)| = \sqrt{ (\frac{\partial f}{\partial x})^2 + (\frac{\partial f}{\partial y})^2}
$$,
and the gradient direction by:
$$
GD = \arctan(\frac{\partial f/\partial y}{\partial f / \partial x})
$$

To adjust for different  


Hints: 
1. You may use the function np.arctan() to calculate $GD$
2. The angle returned by the np.arctan() function is measured in radian, you may turn it into the degree measure by using the function np.rad2deg()
3. It is normal if you encounter the "divide by zero" warning, as $\frac{\partial f}{\partial x}$ can be 0 if the gradient is pointing towards the y-axis.

In [ ]:
GM =  #Calculate the gradient magnitude, use np.sqrt() for square root calculation


In [ ]:
GD =  #Calculate the gradient direction
GD =  #Convert from radien to degree

## Plotting image gradient vector field

All the codes under this section will be given to the students

In [ ]:
#plt.figure(figsize=(12,12)) #change the number for changing the figure size
plt.quiver(dx, dy, color='r')
plt.imshow(small_img, cmap='gray')
plt.title('Image gradient')
#plt.savefig('Image_gradient.jpg',dpi=1000)
plt.show()

In [ ]:
plt.quiver(dx, dy, color='black')
plt.title('Image gradient vector field')
plt.savefig('image_gradient_vector.jpg')
plt.show()

## Plotting Histogram of Oriented Gradients (HOG)

All the codes under this section will be given to the students

In [ ]:
GM = GM.ravel()
GD = GD.ravel()

GD = 90 - GD
GM = GM[~np.isnan(GD)]
GD = GD[~np.isnan(GD)]


GD = GD.reshape(-1)
GM = GM.reshape(-1)

ceil = np.ceil(GD / 20).astype('int32')
floor = np.floor(GD / 20).astype('int32')


w1 = (GD / 20) - floor
w2 = ceil - (GD / 20)

h1 =  np.bincount(floor, GM * w2, minlength=9)  
h2 = np.bincount(ceil, GM * w1, minlength=9)

In [ ]:
h = h1 + h2
h[0] = h[-1]
h = h[:-1]

norm = np.linalg.norm(h)
h_norm = h/norm

values = np.unique(np.floor(GD / 20).astype(np.int64))[:-1]
ticklabel = values*20 - 90
plt.title('Histogram of Oriented Gradients (HOG)')
plt.ylabel('Gradient magnitude')
plt.xlabel('Gradient direction (degree)')
plt.bar(values, h_norm, tick_label=ticklabel)
plt.savefig('HOG.jpg')
plt.show()

# Questions

1. Look at the image gradient vector field plot, in which direction most arrows are pointing at? Can you explain why?
2. How does the HOG plot look like? i.e. Which gradient direction the gradient magnitude peaks at? Why?
3. Try to rotate the orginally image by 90 degress using the function np.rot90() and answer question 1,2 again.